# 13: Proyecto Ejemplo de Dibujo de Grafos por Fuerzas con OOP

-   **Autor**: [Dr. Mario Abarca](https://www.knkillname.org/)
-   **Objetivo**: Ilustrar la aplicación de la Programación Orientada a Objetos (OOP) y otros conceptos del curso en un proyecto integrador: una simulación de dibujo de grafos dirigido por fuerzas, que sirva como modelo y guía para los proyectos finales de los estudiantes.

<a href="https://colab.research.google.com/github/knkillname/uaem.notas.introcomp/blob/master/cuadernos/13.EjemplodeProyectoFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hoy nos sumergiremos de lleno en el desarrollo de un proyecto de ejemplo: una simulación para dibujar grafos de forma *bonita* usando fuerzas de atracción y repulsión. El objetivo es que vean en acción cómo los conceptos que hemos aprendido, especialmente la Programación Orientada a Objetos (OOP), se unen para crear algo interesante y visual, y que esto les sirva de inspiración y guía para sus propios proyectos finales.

## 1. Introducción al Proyecto: Dibujo de Grafos por Fuerzas

La visualización de grafos es fundamental en muchas áreas. Un grafo bien dibujado puede revelar patrones, clústeres y la estructura general de las relaciones que representa. El **dibujo de grafos dirigido por fuerzas** es una técnica popular que utiliza una analogía física para lograr disposiciones estéticas.

**La Idea Intuitiva: Nodos como Partículas, Aristas como Resortes**

Imagina que los nodos (o vértices) de un grafo son pequeñas partículas cargadas eléctricamente que se repelen entre sí. Si estuvieran sueltas, se alejarían unas de otras lo más posible. Ahora, imagina que las aristas (o enlaces) que conectan algunos de estos nodos son resortes. Estos resortes intentan mantener a los nodos que conectan a una "distancia ideal": si los nodos están muy separados, el resorte los jala para acercarlos; si están muy juntos, el resorte los empuja para separarlos (o, más comúnmente, la fuerza de atracción disminuye y la repulsión entre nodos domina).

El objetivo es encontrar una configuración de las posiciones de los nodos en un plano (usualmente 2D) donde este sistema de fuerzas esté en un estado de equilibrio (o cercano a él).

Este proyecto nos permitirá aplicar OOP para modelar los componentes del sistema y usar NumPy para cálculos vectoriales y Matplotlib para la visualización (conceptos de la [Clase 11: Cómputo Científico](https://github.com/knkillname/uaem.notas.introcomp/blob/master/cuadernos/11.C%C3%B3mputoCient%C3%ADfico.ipynb)).

### 🗺️ Hoja de Ruta del Cuaderno

Para que este proyecto sea fácil de digerir, lo construiremos pieza por pieza:

1.  **Entender la Física**: Breve repaso de las fuerzas que usaremos.
2.  **Paso 1: El Nodo**: Crearemos la clase que representa un punto en el espacio.
3.  **Paso 2: La Arista**: Crearemos la clase que conecta dos nodos y genera atracción.
4.  **Paso 3: La Simulación Manual**: Haremos un pequeño experimento sin clases complejas para ver cómo se mueven los nodos.
5.  **Paso 4: El Simulador (OOP)**: Empaquetaremos todo en una clase `SimuladorGrafo` profesional.
6.  **Paso 5: Visualización**: Veremos el resultado final con una animación.

## 2. La Física del Grafo: Fuerzas y Movimiento ⚛️

### 2.1. Las Dos Fuerzas Principales 🤜🤛

Para que nuestro grafo se dibuje solo, vamos a simular un sistema físico simple. Imagina que los nodos son imanes y las aristas son resortes.

| Tipo de Fuerza | ¿Quién la siente? | ¿Qué hace? | Analogía |
| :--- | :--- | :--- | :--- |
| **Repulsión** 🔴 | **Todos** contra todos | Empuja a los nodos lejos unos de otros para que no se amontonen. | Imanes con el mismo polo enfrentados. |
| **Atracción** 🟢 | Solo nodos **conectados** | Jala a los nodos conectados para que se mantengan cerca. | Un resorte elástico uniendo dos pelotas. |

#### 📐 Las Fórmulas (Simplificadas)

1.  **Repulsión (Ley de Coulomb-ish):**
    $$ F_{rep} = \frac{k^2}{distancia} $$
    *Más cerca = Empujón más fuerte.*

2.  **Atracción (Ley de Hooke):**
    $$ F_{atr} = \frac{distancia^2}{k} $$
    *Más lejos = Jalón más fuerte.*

*(Donde $k$ es una constante que define la distancia ideal entre nodos).*

### 2.2. El Ciclo de la Vida (del Grafo) 🔄

El algoritmo funciona repitiendo estos pasos una y otra vez hasta que el grafo se "calma" (equilibrio):

1.  **💥 Calcular Repulsión:** Cada nodo empuja a *todos* los demás.
2.  **🪢 Calcular Atracción:** Cada arista intenta juntar a sus dos extremos.
3.  **🚀 Mover Nodos:** Cada nodo se mueve un poquito en la dirección de la fuerza total que siente.
4.  **🧊 Enfriar:** Reducimos la "temperatura" (cuánto se pueden mover los nodos) para que el dibujo se estabilice y deje de vibrar.

Este ciclo permite que el grafo "se relaje" en una configuración que minimiza la energía del sistema, resultando en un dibujo claro y estético.

## 3. Construyendo la Simulación Paso a Paso 🧱

En lugar de escribir todo el código de golpe, vamos a construir nuestro simulador pieza por pieza. Esto nos ayudará a entender qué hace cada parte y verificar que funcione antes de pasar a la siguiente.

### 3.1. Importaciones Necesarias

Primero, importemos las librerías que usaremos. `numpy` para los vectores y `matplotlib` para dibujar.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import math
# Para animación interactiva en Colab:
from IPython.display import display, clear_output
import time 

### 3.2. Paso 1: La Clase `Nodo` 📍

Un nodo es la unidad básica de nuestro grafo. Necesita saber:
1.  **Quién es**: Su identificador (`id`).
2.  **Dónde está**: Su posición $(x, y)$.
3.  **Qué fuerzas lo empujan**: Un vector de fuerza acumulada.

Y necesita saber hacer dos cosas:
1.  **Recibir empujones**: Sumar fuerzas a su acumulador (`aplicar_fuerza`).
2.  **Moverse**: Cambiar su posición según la fuerza total y luego "descansar" (resetear la fuerza a cero) (`actualizar_posicion`).

In [ ]:
class Nodo:
    def __init__(self, id_nodo, x_inicial, y_inicial):
        self.id = id_nodo
        self.posicion = np.array([float(x_inicial), float(y_inicial)])
        self.fuerza_acumulada = np.zeros(2) # [fx, fy]

    def aplicar_fuerza(self, vector_fuerza):
        self.fuerza_acumulada += vector_fuerza

    def actualizar_posicion(self, factor_movimiento_maximo):
        magnitud_fuerza = np.linalg.norm(self.fuerza_acumulada)
        if magnitud_fuerza > 0: 
            desplazamiento = (self.fuerza_acumulada / magnitud_fuerza) * min(magnitud_fuerza, factor_movimiento_maximo)
            self.posicion += desplazamiento
        self.fuerza_acumulada = np.zeros(2) # Resetear fuerzas para la siguiente iteración

    def __str__(self):
        return f"Nodo({self.id}, Pos: [{self.posicion[0]:.2f}, {self.posicion[1]:.2f}])"

    def __repr__(self):
        return self.__str__()

#### 🧪 Probando el Nodo

Vamos a crear un nodo y empujarlo manualmente para ver si funciona.

In [ ]:
nodo_A_ej = Nodo("NodoA", 1.0, 2.0)
nodo_A_ej 

**Aplicar una fuerza:**

In [ ]:
fuerza_1_ej = np.array([1.0, -0.5])
nodo_A_ej.aplicar_fuerza(fuerza_1_ej)
nodo_A_ej.fuerza_acumulada

**Aplicar otra fuerza:**

In [ ]:
fuerza_2_ej = np.array([-0.3, 0.7])
nodo_A_ej.aplicar_fuerza(fuerza_2_ej)
nodo_A_ej.fuerza_acumulada

**Actualizar posición:**

In [ ]:
factor_desplazamiento_ej = 0.1
print(f"Posición de {nodo_A_ej.id} (antes de actualizar): {nodo_A_ej.posicion}")
nodo_A_ej.actualizar_posicion(factor_desplazamiento_ej)
print(f"Posición de {nodo_A_ej.id} (después de actualizar): {nodo_A_ej.posicion}")
print(f"Fuerza acumulada en {nodo_A_ej.id} (después de actualizar): {nodo_A_ej.fuerza_acumulada}")
nodo_A_ej

### 3.3. Paso 2: La Clase `Arista` 🔗

Una arista conecta dos nodos. Funciona como un **resorte**:
- Si los nodos están muy lejos, los atrae.
- Si están muy cerca, los repele (aunque en nuestro modelo simplificado, la repulsión principal vendrá de otra parte).

La arista necesita saber:
1.  **A quién conecta**: `nodo_origen` y `nodo_destino`.
2.  **Su longitud ideal**: A qué distancia le gustaría que estuvieran los nodos.
3.  **Su fuerza**: Qué tan rígido es el resorte (`k_atraccion`).

Su trabajo es calcular la fuerza y aplicársela a *ambos* nodos (en direcciones opuestas).

In [ ]:
class Arista:
    def __init__(self, nodo_origen, nodo_destino, longitud_ideal=1.5, k_atraccion=0.05):
        # nodo_origen y nodo_destino son objetos de la clase Nodo
        self.nodo_origen = nodo_origen
        self.nodo_destino = nodo_destino
        self.longitud_ideal = longitud_ideal
        self.k_atraccion = k_atraccion # Constante del "resorte"

    def calcular_y_aplicar_fuerza_atraccion(self):
        vector_direccion = self.nodo_destino.posicion - self.nodo_origen.posicion
        distancia_actual = np.linalg.norm(vector_direccion)

        if distancia_actual == 0: # Nodos superpuestos, evitar división por cero
            return

        # Fuerza de resorte (Ley de Hooke): F = k * (distancia_actual - longitud_ideal)
        fuerza_magnitud = self.k_atraccion * (distancia_actual - self.longitud_ideal)
        
        direccion_normalizada = vector_direccion / distancia_actual
        fuerza_vectorial = direccion_normalizada * fuerza_magnitud
        
        self.nodo_origen.aplicar_fuerza(fuerza_vectorial)
        self.nodo_destino.aplicar_fuerza(-fuerza_vectorial) # Fuerza opuesta

    def __str__(self):
        return f"Arista({self.nodo_origen.id} - {self.nodo_destino.id})"
    
    def __repr__(self):
        return self.__str__()

#### 🧪 Probando la Arista

Vamos a conectar dos nodos y ver cómo la arista genera fuerzas sobre ellos.

In [ ]:
nodo_P_ej = Nodo("P", 0.0, 0.0)
nodo_Q_ej = Nodo("Q", 3.0, 4.0) # Distancia inicial es 5
# Supongamos longitud_ideal = 3.0 y k_atraccion = 0.1
arista_PQ_ej = Arista(nodo_P_ej, nodo_Q_ej, longitud_ideal=3.0, k_atraccion=0.1)

print(f"Nodo inicial P: {nodo_P_ej}")
print(f"Nodo inicial Q: {nodo_Q_ej}")
print(f"Arista creada: {arista_PQ_ej}")

**Calcular y aplicar fuerza de atracción:**

In [ ]:
print(f"Fuerza en Nodo P (antes de atracción): {nodo_P_ej.fuerza_acumulada}")
print(f"Fuerza en Nodo Q (antes de atracción): {nodo_Q_ej.fuerza_acumulada}")

arista_PQ_ej.calcular_y_aplicar_fuerza_atraccion()

# Explicación del cálculo de la fuerza aplicada:
# Distancia actual entre P y Q es 5.0. Longitud ideal es 3.0. k_atraccion es 0.1.
# Magnitud de la fuerza: 0.1 * (5.0 - 3.0) = 0.2.
# Dirección normalizada de P hacia Q: (3/5, 4/5) = (0.6, 0.8).
# Fuerza vectorial sobre P (hacia Q): 0.2 * (0.6, 0.8) = (0.12, 0.16).
# Fuerza vectorial sobre Q (hacia P): - (0.12, 0.16) = (-0.12, -0.16).
print(f"Fuerza en Nodo P (después de atracción): {nodo_P_ej.fuerza_acumulada}") 
print(f"Fuerza en Nodo Q (después de atracción): {nodo_Q_ej.fuerza_acumulada}")

### 3.4. Paso 3: Simulador Manual (Sin Clases Complejas) 🛠️

Antes de crear una clase gigante que maneje todo, intentemos hacer una simulación "a mano" con un bucle `for`. Esto nos ayudará a entender la lógica del algoritmo:

1.  Calcular fuerzas de repulsión (todos se odian).
2.  Calcular fuerzas de atracción (los amigos se quieren).
3.  Mover nodos.
4.  Repetir.

In [ ]:
# Crear dos nodos
n1 = Nodo("1", 0, 0)
n2 = Nodo("2", 0.5, 0) # Muy cerca!

# Crear una arista
arista = Arista(n1, n2, longitud_ideal=2.0, k_atraccion=0.1)

print("--- Inicio ---")
print(n1)
print(n2)

# Simular 5 pasos
for i in range(5):
    # 1. Repulsión (simplificada para este ejemplo manual)
    dist = np.linalg.norm(n1.posicion - n2.posicion)
    fuerza_rep = 0.5 / (dist**2) # Fuerza arbitraria
    vec_dir = (n1.posicion - n2.posicion) / dist
    
    n1.aplicar_fuerza(vec_dir * fuerza_rep)
    n2.aplicar_fuerza(-vec_dir * fuerza_rep)
    
    # 2. Atracción
    arista.calcular_y_aplicar_fuerza_atraccion()
    
    # 3. Mover
    n1.actualizar_posicion(0.1)
    n2.actualizar_posicion(0.1)
    
    print(f"Paso {i+1}: Distancia {np.linalg.norm(n1.posicion - n2.posicion):.2f}")

### 3.5. Intermedio: ¿Cómo dibujamos esto? 🎨

Hasta ahora solo hemos visto números en la consola. Pero queremos ver **bolitas y palitos**.

Aunque existen librerías como `networkx` que hacen esto automático, para entenderlo a fondo (y porque somos valientes), vamos a dibujarlo nosotros mismos usando `matplotlib`.

Es muy fácil:
1.  **Nodos:** Son puntos en un plano. Usamos `plt.scatter(x, y)`.
2.  **Aristas:** Son líneas que unen dos puntos. Usamos `plt.plot([x1, x2], [y1, y2])`.

**Ejemplo Práctico:** Vamos a dibujar un mapa simplificado de carreteras entre algunos estados del centro de México.

In [ ]:
# Definimos las coordenadas (aproximadas) de los estados
posiciones = {
    "CDMX": np.array([0.0, 0.0]),
    "Morelos": np.array([0.2, -0.8]),
    "Puebla": np.array([1.5, -0.5]),
    "EdoMex": np.array([-0.8, 0.3]),
    "Querétaro": np.array([-1.0, 1.5]),
    "Guerrero": np.array([0.0, -2.0])
}

# Definimos las carreteras (conexiones)
carreteras = [
    ("CDMX", "Morelos"), ("CDMX", "Puebla"), ("CDMX", "EdoMex"),
    ("EdoMex", "Querétaro"), ("EdoMex", "Morelos"), ("EdoMex", "Puebla"),
    ("Morelos", "Guerrero"), ("Puebla", "Morelos")
]

plt.figure(figsize=(6, 6))

# 1. Dibujar las Carreteras (Aristas)
for origen, destino in carreteras:
    p1 = posiciones[origen]
    p2 = posiciones[destino]
    # Dibujamos una línea entre p1 y p2
    plt.plot([p1[0], p2[0]], [p1[1], p2[1]], color='gray', linestyle='--', zorder=1)

# 2. Dibujar los Estados (Nodos)
xs = [pos[0] for pos in posiciones.values()]
ys = [pos[1] for pos in posiciones.values()]
plt.scatter(xs, ys, s=1000, c='orange', edgecolors='brown', zorder=2)

# 3. Ponerles nombre
for nombre, pos in posiciones.items():
    plt.text(pos[0], pos[1], nombre, ha='center', va='center', fontsize=9, fontweight='bold')

plt.title("Mapa de Carreteras Centro de México")
plt.axis('equal') # Para que no se deforme el mapa
plt.grid(True, alpha=0.2)
plt.show()

### 3.6. Paso 4: El Simulador Completo (OOP) 🏗️

Ahora que entendemos la lógica básica (repulsión + atracción + movimiento), vamos a empaquetar todo en una clase `SimuladorGrafo`. Esta clase se encargará de:
1.  Guardar todos los nodos y aristas.
2.  Calcular las fuerzas entre *todos* los pares de nodos (no solo 2).
3.  Dibujar el grafo usando Matplotlib.

Aquí está el código completo de la clase gestora:

In [ ]:
class SimuladorGrafo:
    def __init__(self, k_repulsion_base=0.1, factor_movimiento_inicial=0.1, factor_enfriamiento=0.99, area_total_dibujo=10.0):
        self.nodos = {} # Usaremos un diccionario para acceder a nodos por ID fácilmente
        self.aristas = []
        self.k_repulsion_base = k_repulsion_base # Factor base para la repulsión
        self.factor_movimiento = factor_movimiento_inicial
        self.factor_enfriamiento = factor_enfriamiento
        self.area_total_dibujo = area_total_dibujo
        self.k_optimo_distancia = 1.0 # Distancia óptima entre nodos, se calcula después

    def agregar_nodo(self, id_nodo, x_inicial=None, y_inicial=None):
        if x_inicial is None: # Posiciones aleatorias si no se especifican
            x_inicial = (np.random.rand() - 0.5) * self.area_total_dibujo * 0.5
        if y_inicial is None:
            y_inicial = (np.random.rand() - 0.5) * self.area_total_dibujo * 0.5
        
        if id_nodo not in self.nodos:
            nodo = Nodo(id_nodo, x_inicial, y_inicial)
            self.nodos[id_nodo] = nodo
            # Recalcular k_optimo_distancia basado en el número de nodos (Fruchterman-Reingold)
            if len(self.nodos) > 0:
                 self.k_optimo_distancia = math.sqrt( (self.area_total_dibujo**2) / len(self.nodos) )
            return nodo
        return self.nodos[id_nodo] # Retorna el nodo existente si el ID ya estaba

    def agregar_arista(self, id_nodo1, id_nodo2, longitud_ideal=None, k_atraccion=0.02):
        if id_nodo1 in self.nodos and id_nodo2 in self.nodos:
            nodo1 = self.nodos[id_nodo1]
            nodo2 = self.nodos[id_nodo2]
            
            if longitud_ideal is None: # Usar k_optimo_distancia como longitud ideal para aristas
                longitud_ideal = self.k_optimo_distancia 
            
            arista = Arista(nodo1, nodo2, longitud_ideal, k_atraccion)
            self.aristas.append(arista)
            return arista
        else:
            raise ValueError(f"Uno o ambos nodos ({id_nodo1}, {id_nodo2}) no existen en el grafo.")

    def _calcular_fuerzas_repulsion(self):
        lista_nodos_obj = list(self.nodos.values())
        for i in range(len(lista_nodos_obj)):
            for j in range(i + 1, len(lista_nodos_obj)): # Evitar pares duplicados y auto-repulsión
                nodo1 = lista_nodos_obj[i]
                nodo2 = lista_nodos_obj[j]
                
                vector_direccion = nodo1.posicion - nodo2.posicion
                distancia = np.linalg.norm(vector_direccion)
                
                if distancia == 0: # Nodos superpuestos
                    fuerza_vectorial_rep = (np.random.rand(2) - 0.5) * 0.01 
                else:
                    fuerza_magnitud_rep = self.k_repulsion_base * (self.k_optimo_distancia**2) / distancia
                    direccion_normalizada_rep = vector_direccion / distancia
                    fuerza_vectorial_rep = direccion_normalizada_rep * fuerza_magnitud_rep
                
                nodo1.aplicar_fuerza(fuerza_vectorial_rep)
                nodo2.aplicar_fuerza(-fuerza_vectorial_rep) # Fuerza opuesta

    def _calcular_fuerzas_atraccion(self):
        for arista in self.aristas:
            arista.calcular_y_aplicar_fuerza_atraccion()

    def _actualizar_posiciones_nodos(self):
        for nodo in self.nodos.values():
            nodo.actualizar_posicion(self.factor_movimiento)

    def paso_simulacion(self):
        self._calcular_fuerzas_repulsion()
        self._calcular_fuerzas_atraccion()
        self._actualizar_posiciones_nodos()
        self.factor_movimiento *= self.factor_enfriamiento

    def dibujar(self, titulo_iteracion='', ax=None, clear_ax=True):
        standalone_plot = ax is None # Determina si se debe crear una figura o usar una existente
        if standalone_plot:
            # Si no se proporciona un 'ax', crea una nueva figura y eje.
            fig, ax = plt.subplots(figsize=(8, 8)) # Ajustar tamaño según necesidad
        
        if clear_ax:
            ax.clear() # Limpiar el eje para la nueva iteración
        
        # Dibujar aristas primero para que estén detrás de los nodos
        for arista in self.aristas:
            pos_origen = arista.nodo_origen.posicion
            pos_destino = arista.nodo_destino.posicion
            ax.plot([pos_origen[0], pos_destino[0]], 
                    [pos_origen[1], pos_destino[1]], 
                    'k-', alpha=0.3, zorder=1) # Líneas negras semitransparentes

        # Dibujar nodos
        if self.nodos: # Solo si hay nodos
            x_coords = [nodo.posicion[0] for nodo in self.nodos.values()]
            y_coords = [nodo.posicion[1] for nodo in self.nodos.values()]
            ax.scatter(x_coords, y_coords, s=300, c='skyblue', edgecolors='black', zorder=2, alpha=0.8)
            
            # Etiquetar nodos
            for nodo_id, nodo_obj in self.nodos.items():
                ax.text(nodo_obj.posicion[0], nodo_obj.posicion[1], str(nodo_id), 
                        ha='center', va='center', fontsize=9, color='black', zorder=3)
        
        ax.set_title(f"Dibujo de Grafo por Fuerzas - {titulo_iteracion}")
        ax.set_xticks([]) # Ocultar ejes
        ax.set_yticks([])
        
        # Establecer límites para centrar el grafo y evitar que se salga demasiado
        if self.nodos:
            all_x = [n.posicion[0] for n in self.nodos.values()]
            all_y = [n.posicion[1] for n in self.nodos.values()]
            if all_x and all_y: # Asegurarse de que no estén vacías
                min_x, max_x = min(all_x), max(all_x)
                min_y, max_y = min(all_y), max(all_y)
                padding_x = (max_x - min_x) * 0.15 if (max_x - min_x) > 1e-6 else 1.0
                padding_y = (max_y - min_y) * 0.15 if (max_y - min_y) > 1e-6 else 1.0
                ax.set_xlim([min_x - padding_x, max_x + padding_x])
                ax.set_ylim([min_y - padding_y, max_y + padding_y])
        else: # Límites por defecto si no hay nodos
             ax.set_xlim([-self.area_total_dibujo/2, self.area_total_dibujo/2])
             ax.set_ylim([-self.area_total_dibujo/2, self.area_total_dibujo/2])

        ax.set_aspect('equal', adjustable='box') # Mantener la proporción de aspecto
        
        # Solo llamar a plt.show() si la figura se creó dentro de este método
        if standalone_plot: 
            plt.show()

In [ ]:
sim_basico_ej = SimuladorGrafo(factor_movimiento_inicial=0.5, area_total_dibujo=5.0)

nodo_s1_ej = sim_basico_ej.agregar_nodo("S1", 0, 0)
nodo_s2_ej = sim_basico_ej.agregar_nodo("S2", 1, 1)
nodo_s3_ej = sim_basico_ej.agregar_nodo("S3", -1, 1)

arista1_ej = sim_basico_ej.agregar_arista("S1", "S2")
arista2_ej = sim_basico_ej.agregar_arista("S1", "S3")

**Mostrar información básica del simulador:**

In [ ]:
print("Nodos en el simulador básico:")
for nid, n_obj in sim_basico_ej.nodos.items(): print(n_obj)
    
print("\nAristas en el simulador básico:")
for ar in sim_basico_ej.aristas: print(ar)
    
print(f"\nDistancia óptima k (básico): {sim_basico_ej.k_optimo_distancia:.2f}")

**Dibujar el estado inicial:**

In [ ]:
# La función dibujar llamará a plt.show() si no se le pasa un 'ax'
sim_basico_ej.dibujar("Estado Inicial (Básico)") 

**Realizar un paso de simulación y mostrar estado:**

In [ ]:
print("\nRealizando un paso de simulación (básico)...")
sim_basico_ej.paso_simulacion()

print("\nNodos después de 1 paso (básico):")
for n_id, n_obj in sim_basico_ej.nodos.items(): print(n_obj)
print(f"Factor de movimiento (básico): {sim_basico_ej.factor_movimiento:.3f}")

# Dibujar después de un paso
sim_basico_ej.dibujar("Después de 1 Paso (Básico)") 

### 3.7. Paso 5: Visualización y Animación 🎥

**Configuración del Grafo de Prueba:**

In [ ]:
simulador_completo = SimuladorGrafo(
    k_repulsion_base=0.2, 
    factor_movimiento_inicial=0.2, 
    factor_enfriamiento=0.99,
    area_total_dibujo=8.0
)

simulador_completo.agregar_nodo('A')
simulador_completo.agregar_nodo('B')
simulador_completo.agregar_nodo('C')
simulador_completo.agregar_nodo('D')
simulador_completo.agregar_nodo('E')

simulador_completo.agregar_arista('A', 'B', k_atraccion=0.03)
simulador_completo.agregar_arista('B', 'C', k_atraccion=0.03)
simulador_completo.agregar_arista('C', 'D', k_atraccion=0.03)
simulador_completo.agregar_arista('D', 'A', k_atraccion=0.03) # Ciclo
simulador_completo.agregar_arista('A', 'C', k_atraccion=0.01)
simulador_completo.agregar_arista('B', 'D', k_atraccion=0.01) # Diagonales
simulador_completo.agregar_arista('E', 'A', k_atraccion=0.03)
simulador_completo.agregar_arista('E', 'C', k_atraccion=0.03)

**Estado Inicial:**

In [ ]:
# Creamos la figura y el eje una vez para la "animación"
fig_anim, ax_anim = plt.subplots(figsize=(7, 7)) 
# Pasamos ax_anim a dibujar, y clear_ax=False para el primer dibujo para no borrar si ya había algo.
simulador_completo.dibujar(titulo_iteracion="Estado Inicial Completo", ax=ax_anim, clear_ax=False) 

**¡Acción! (Ejecutar Simulación):**

In [ ]:
num_iteraciones = 500
print(f"k_optimo_distancia calculado para simulación completa: {simulador_completo.k_optimo_distancia:.2f}")

for i in range(num_iteraciones):
    simulador_completo.paso_simulacion()
    if (i + 1) % 10 == 0: # Redibujar cada 10 iteraciones
        clear_output(wait=True) # Limpiar la salida de la celda anterior
        simulador_completo.dibujar(titulo_iteracion=f"Iteración {i+1}", ax=ax_anim, clear_ax=True) # Reutilizar y limpiar el mismo eje
        display(fig_anim) # Mostrar la figura actualizada
        time.sleep(0.1) # Pequeña pausa para que se vea la actualización

**Resultado Final:**

In [ ]:
clear_output(wait=True) # Limpiar la última salida del bucle
simulador_completo.dibujar(titulo_iteracion=f"Final ({num_iteraciones} iteraciones)", ax=ax_anim, clear_ax=True)
display(fig_anim) # Mostrar la figura final

print(f"\nFactor de movimiento final: {simulador_completo.factor_movimiento:.4f}")
print("Estado final de los nodos:")
for nodo_id, nodo_obj in simulador_completo.nodos.items():
    print(nodo_obj)
plt.close(fig_anim) # Cerrar la figura para liberar memoria

**Nota sobre la animación en Colab/Jupyter:** El código ahora utiliza `IPython.display.clear_output(wait=True)` y `IPython.display.display(fig)` para crear una animación actualizando la misma figura en la celda. Esto es generalmente más robusto para Colab. Para animaciones más complejas o para exportar a video/GIF, se podría usar `matplotlib.animation.FuncAnimation`.

## 6. Autoevaluación y Conclusiones: Tu Camino Autodidacta 📝

### 6.1. La "Confesión" y tu Checklist de Aprendizaje 🕵️‍♂️

A ver, seamos honestos. **¿Era estrictamente necesario usar Programación Orientada a Objetos (OOP) para mover unos cuantos puntos en la pantalla?**

La respuesta corta es: **No**. Probablemente con unos cuantos arrays de NumPy y un bucle `while` hubiéramos terminado en la mitad de líneas de código.

Pero... **estás aquí para aprender**. Este curso está diseñado para estudiantes autodidactas que quieren dominar las herramientas, no solo usarlas. Usamos OOP para mostrarte cómo estructurar un problema "real" de forma **ordenada, modular y escalable**. Si mañana quieres agregar gravedad, solo modificas la clase `Nodo`. ¡Esa es la magia!

Como este curso es ahora un recurso abierto en GitHub, **no hay calificaciones ni profesor persiguiéndote**. Tú eres tu propio maestro. La [Rúbrica del Proyecto Final](../recursos/rubricaDeProyectoFinal.md) ya no es una hoja de examen, sino tu **mapa de tesoro** para saber si realmente dominas los temas.

Veamos cómo se compara este ejemplo con ese mapa:

| Habilidad Clave | ¿Lo usamos aquí? | Reto para tu propio proyecto 🚀 |
| :--- | :---: | :--- |
| **Estructura y Orden** | ✅ | Mantén tu código limpio. Tu "yo del futuro" te lo agradecerá. |
| **Python Básico** | ✅ | Variables, funciones, f-strings... asegúrate de fluir con esto. |
| **Lógica e Iteración** | ⚠️ | Usamos muchos `for`, pero **no usamos Recursividad**. ¿Podrías implementarla tú? (ej. recorrer un árbol). |
| **Estructuras de Datos** | ✅ | Diccionarios y listas son el pan de cada día. ¡Domínalos! |
| **Archivos (I/O)** | ❌ | **Faltante importante.** Este ejemplo crea datos al vuelo. **Tu reto:** Carga los nodos desde un CSV o JSON. |
| **OOP (Clases)** | ⭐ | ¡Sobrado! Clases `Nodo`, `Arista`, `SimuladorGrafo`. Intenta crear tus propias clases. |
| **Bibliotecas (NumPy/Matplotlib)** | ✅ | Usamos NumPy para vectores y Matplotlib para dibujar. Son herramientas esenciales en ciencia de datos. |

**Tu Misión:** Este cuaderno te da una base sólida (digamos, un 80% del camino). Para completar tu aprendizaje, te reto a clonar este repo y agregarle la lectura de archivos o una interfaz interactiva. ¡El límite lo pones tú!

### 6.2. Discusiones y Siguientes Pasos 🗣️

Ahora que has construido tu propio motor de física, es momento de ampliar horizontes. Usa tu asistente de IA favorito (ChatGPT, Claude, Copilot) para explorar estos temas y profundizar tu aprendizaje:

**Discusión 1: Eficiencia y Escalamiento 🚀**
El algoritmo que implementamos compara todos los nodos contra todos los nodos para la repulsión, lo que tiene una complejidad de $O(V^2)$.
*   **Pregunta a tu IA:** *"¿Qué algoritmos existen para dibujar grafos con millones de nodos de manera eficiente? Explícame cómo funciona el algoritmo **Barnes-Hut** y cómo reduce la complejidad del cálculo de fuerzas."*

**Discusión 2: Herramientas Profesionales 🛠️**
No siempre querrás escribir 200 líneas de código para ver un grafo.
*   **Pregunta a tu IA:** *"Hazme una tabla comparativa entre **NetworkX**, **Graphviz** y **PyVis** para visualización de grafos en Python. ¿Cuál debería usar si quiero interactividad en una página web y cuál para publicaciones científicas estáticas?"*

**Discusión 3: Análisis de Redes 🕸️**
Dibujar el grafo es solo el principio. Lo interesante es analizar su estructura.
*   **Pregunta a tu IA:** *"¿Qué es el análisis de redes sociales (SNA)? Explícame qué significan métricas como 'Grado' (Degree), 'Centralidad de Intermediación' (Betweenness Centrality) y 'PageRank', y cómo podría calcularlas usando mi clase `SimuladorGrafo` o `NetworkX`."*

---

### Palabras Finales: Construye tu Portafolio 💼

Este proyecto fue un ejercicio de **"abrir el capó"**. Al construirlo desde cero, ganaste el superpoder de entender que no es magia, son solo vectores y matemáticas.

**Consejos para tu camino autodidacta:**
1.  **Crea, no solo consumas:** No te limites a leer estos cuadernos. Escribe código, rompe cosas y arréglalas.
2.  **Publica tu trabajo:** Sube tus versiones de estos proyectos a tu propio GitHub. Un portafolio con proyectos explicados vale más que mil certificados.
3.  **Diviértete:** Elijan temas que les apasionen (videojuegos, finanzas, biología). La curiosidad es el mejor combustible.

¡Mucho éxito en tu viaje de aprendizaje! 🚀